In [22]:
import sys
print(sys.path)

import os

# get the current working directory
current_working_directory = os.getcwd()

# print output to the console
print(current_working_directory)

['/workspaces/onboard/app', '/dependencies', '/usr/local/lib/python312.zip', '/usr/local/lib/python3.12', '/usr/local/lib/python3.12/lib-dynload', '', '/home/anvil/.local/lib/python3.12/site-packages', '/usr/local/lib/python3.12/site-packages']
/workspaces/onboard/app


In [1]:
import importlib

def load_class(module_name, class_name):
    module = importlib.import_module(f"{module_name}.{class_name}")
    cls = getattr(module, ''.join(word.title() for word in class_name.split('_')) )
    instance = cls()
    return instance


# Example usage:
instance = load_class("processors", "cafe_hayek")
if instance:
    result = instance.process("hello world")
    print(result)


hello world


In [ ]:
import random

random_number = random.randint(5001, 9998)
print("Random number:", random_number)

Random number: 9830


In [5]:
async def nested():
    return 42

async def main():

    # Let's do it differently now and await it:
    print(await nested())  # will print "42".
    
await main()

42


In [5]:
from bs4 import BeautifulSoup
import html
import re
    
def clean_html(text):
  text = text.replace('\n', ' ').replace('\r', ' ').strip()
  
  
  
  print(text)
  print("===============================================================")
  
  text = BeautifulSoup(html.unescape(text), 'lxml').text
  text = re.sub(r'\[.*?\].*$', '', text)
  # text = re.sub(r'http[s]?://\S+', '', text, flags=re.IGNORECASE)
  # text = ' '.join([x.capitalize() for x in text.split(' ')])

  return text.strip()

clean_html("")

''

In [2]:
import aiohttp
import asyncio

import time
#from utils import clean_html, copy_default_to_configs, load_file
import feedparser
#from post_processor import post_processor

async def load_feed(widget, column):
  start_time = time.time()
  async with aiohttp.ClientSession() as session:
    async with session.get(widget['url']) as response:
      parsed_feed = feedparser.parse(await response.text())
      widget['summary_enabled'] = widget.get('summary_enabled', True)
      widget['articles'] = [{
          'title': " ".join(entry.get('title', 'No Title').split()).strip() , 
          'link': entry.link, 
          'summary': clean_html(entry.get('summary', ''))} for entry in parsed_feed.entries[:10]] if 'entries' in parsed_feed else []
      widget = post_processor.process(widget['name'], widget)
      column.append(widget)
      return (time.time() - start_time)

async def index(tab_name=None):
  start_time = time.time()
  # Load feeds and bookmarks
  layout = load_file('layout.yml')
  headers = layout['headers']
  
  tabs = layout['tabs']
  if tab_name is None:
    tab = tabs[0]
  else:
    tab = next((tab for tab in tabs if tab["name"].lower() == tab_name.lower()), tabs[0])
  current_tab = tab['name']
  
  column_count = tab['columns']
  columns = [[] for _ in range(column_count)]
  
  tasks = []
  
  print(f"loading => {(time.time() - start_time)} seconds")
  
  start_time = time.time()\
  # Add feeds to the appropriate column
  if tab['widgets']:
    for widget in tab['widgets']:
      column_index = (widget['column'] - 1) % column_count
      if widget['type'] == 'feed':
        task = asyncio.create_task(load_feed(widget, columns[column_index]))
        task.add_done_callback(lambda t: print(f'{t.result()}'))
        tasks.append(task)
      elif widget['type'] == 'bookmarks':
        widget['articles'] = [{'title': entry['title'], 'link': entry['url']} for entry in widget['bookmarks']]
        columns[column_index].append(widget) 
  
  print(f"looping => {(time.time() - start_time)} seconds")
  start_time = time.time()      
  await asyncio.wait(tasks)
  print(f"waiting => {(time.time() - start_time)} seconds")
  
  result = 0
  for task in tasks:
    result += task.result()
  print(f"total => {result} seconds")
  
  start_time = time.time()
  for column in columns:
    column.sort(key = lambda x: x['position'])
  print(f"sorting => {(time.time() - start_time)} seconds")

await index()

NameError: name 'load_file' is not defined

In [5]:
#from utils import load_file
from types import SimpleNamespace
from yamlpath.common import Parsers
from yamlpath.wrappers import ConsolePrinter
from yamlpath import Processor

logging_args = SimpleNamespace(quiet=True, verbose=False, debug=False)
log = ConsolePrinter(logging_args)
parser = Parsers.get_yaml_editor()

# At this point, you'd load or parse your YAML file, stream, or string.  This
# example demonstrates loading YAML data from an external file.  You could also
# use the same function to load data from STDIN or even a String variable.  See
# the Parser class for more detail.
yaml_file = "../app/configs/layout.yml"
(yaml_data, doc_loaded) = Parsers.get_yaml_data(parser, log, yaml_file)
if not doc_loaded:
    # There was an issue loading the file; an error message has already been
    # printed via ConsolePrinter.
    exit(1)

# Pass the logging facility and parsed YAML data to the YAMLPath Processor
processor = Processor(log, yaml_data)


In [4]:
from yamlpath import YAMLPath
from yamlpath.exceptions import YAMLPathException

yaml_path = YAMLPath("/tabs/*/widgets[name = 'Cafe Hayek']")
try:
    for node_coordinate in processor.get_nodes(yaml_path, return_coordinates=True, return_node=True, mustexist=True):
        print("Got {} from '{}'.".format(node_coordinate, yaml_path))
        # Do something with each node_coordinate.node (the actual data)
except YAMLPathException as ex:
    # If merely retrieving data, this exception may be deemed non-critical
    # unless your later code absolutely depends upon a result.
    log.error(ex)